# Anzeige der Temperaturdaten aus der Heizung aus Temperaturen.db

In [23]:
import os, sys
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Layout, Label
from ipywidgets import interact, interactive, fixed, interact_manual
import time
import datetime as dt
import sqlite3
import numpy as np
import warnings
from dotenv import dotenv_values
#import plotly.express as px   
import plotly.io as pio
pio.renderers.default='iframe'
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.plotting.backend = "plotly"
pd.options.mode.chained_assignment = None 

#wegen IOS:
if sys.platform.startswith('win'):
    from zoneinfo import ZoneInfo
    import paramiko


from scp import SCPClient
DB_FILENAME = 'Temperaturen.db'
DEFAULT_DURATION_DAYS=4
AGE_DIFF_SECONDS = 60*60*24

def createSSHClient(server, port, user, password):
    if not sys.platform.startswith('win'):
        return None
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())   
    client.connect(server, port, user, password)
    return client
    
def get_db_file():
    print('Hole Daten ...', end = '')
    config = dotenv_values(".env")
    ssh = createSSHClient("192.168.202.41", 22, config.get('TEMPDB_USER'), config.get('TEMPDB_PASSWD'))
    if ssh:
        scp = SCPClient(ssh.get_transport())
        scp.get('/var/lib/grafana/' + DB_FILENAME)
        print('Feddisch')

def show_temperatur_file_status(db_filename, agediff):
    st=os.stat(db_filename)    
    mtime=st.st_mtime
    if time.time() - os.path.getmtime(db_filename) > agediff:
        button.style.button_color = '#FF0000'
        print('Die Temperaturdaten sind veraltet: ' + format(dt.datetime.fromtimestamp(mtime)) )
    else:
        button.style.button_color = '#90ee90'
        print('Die Temperaturdaten sind aktuell: ' + format(dt.datetime.fromtimestamp(mtime)) )


In [21]:
def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
    
def plot_dfplot(start, dauer, einheit):
    # Mit Dropdown: dt.datetime
    # Mit DatePickier: datetime.date

    #start_object = dt.datetime.strptime(start, '%Y-%m-%d')
    start_object = dt.datetime(start.year, start.month, start.day)
    
    if einheit == 'Tage':
        end_date = start_object + dt.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_date = start_object + dt.timedelta(hours=dauer)
    else:
        end_date = start_object + dt.timedelta(days=dauer)
    #print('Start: ' + str(start) + ' End: ' + str(end_date))
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_date )]
    subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"], grid=True)

In [22]:
def plot_temperaturverlauf(df, start, dauer, show_spreizung, show_heizungs_einstellungen):
    start_object = dt.datetime(start.year, start.month, start.day,  tzinfo=ZoneInfo("Europe/Berlin"))   
    end_object = start_object + dt.timedelta(hours=dauer)
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object )]
    subset = subset[(subset['UnixTime'] <= end_object)]
    #display(subset['UnixTime'].min)
    #display(subset['UnixTime'].max)
    
    subset.set_index(['UnixTime'])
    if show_spreizung: subset['Spreizung'] = subset['VorlaufTemp'] - subset['RuecklaufTemp'] 
    subset = subset.astype({'WohnzimmerTemp':'float64'})
    subset = subset.astype({'AussenTemp':'float64'})    
    fig_title = "Temperaturverlauf Heizung"
    fig_title = "Temperaturverlauf Heizung: " + subset['UnixTime'].min().strftime('%d.%m.%Y') + " - " + subset['UnixTime'].max().strftime('%d.%m.%Y')

    kurven= ['VorlaufTemp', 'RuecklaufTemp', 'WohnzimmerTemp', 'AussenTemp']
    einstellungs_kurven= ['Steigung', 'Niveau','UPumpe']
    if show_spreizung: kurven.append('Spreizung')
    if show_heizungs_einstellungen: 
        kurven.extend(einstellungs_kurven)        
    #print(str(kurven))
    fig_verlauf = subset.plot(x='UnixTime',y=kurven, title=fig_title,
                     labels={
                     "UnixTime": "Zeit",
                     "VorlaufTemp": "VorlaufTemperatur",
                     "UPumpe": "Einstellung Umwälzpumpe"
                 },)
    fig_verlauf.update_layout(xaxis_title="Zeit", yaxis_title="Temperaturen in °C", legend_title=None)    
    fig_verlauf.show()


def prepare_file():
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')
    df_orig = df
    df.set_index(['UnixTime'])
    return df

def on_button_clicked(b):
    with output:
        get_db_file()
        show_temperatur_file_status(DB_FILENAME, AGE_DIFF_SECONDS)

def get_temperatur_file_status(db_filename, agediff):
    st=os.stat(db_filename)    
    mtime=st.st_mtime
    if time.time() - os.path.getmtime(db_filename) > agediff:
        return (False, 'Temperaturdaten veraltet: ' + format(dt.datetime.fromtimestamp(mtime)) )
    else:
        return (True, 'Temperaturdaten aktuell: ' + format(dt.datetime.fromtimestamp(mtime)))

    
def display_temp_verlauf():
    df = None
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')
        
    df.index = df['UnixTime']
    #print(df.index())
    
    #df2 = df.iloc[df.index.between_time('08:00', '12:00')]
    #df.between_time('8:00','12:00')
    
    #df_orig = df_orig.set_index(['UnixTime'])
    #df_orig.between_time('08:00', '12:00')
    #df= df_orig


    layout_btn = widgets.Layout(width='300px')
    is_too_old, btn_text = get_temperatur_file_status(DB_FILENAME, AGE_DIFF_SECONDS)
    widget_button = widgets.Button(description=btn_text, layout=layout_btn)
    if is_too_old:
        widget_button.style.button_color = '#90ee90'
    else:
        widget_button.style.button_color = '#FF0000'
        
    
 

    #df.info()
    wstyle = {'description_width': 'initial'}
    opts = [('1 Stunde', 1), ('6 Stunden', 6), ('12 Stunden', 12), ('24 Stunden', 24), ('2 Tage', 48),('3 Tage', 72), ('4 Tage', 96), ('7 Tage', 168)]
    widget_dauer = widgets.Dropdown(options=opts,value=96,description='Anzeigedauer:', style=wstyle)
    art_der_kurve = ['Temperaturverlauf', 'Spreizung']
    #widget_dauer = widgets.IntSlider(min=1,max=10,step=1, description="Dauer", value=DEFAULT_DURATION_DAYS)
    #dates_us = df.UnixTime.dt.strftime('%Y-%m-%d').unique()
    #widget_start = widgets.Select(options=np.sort(dates_us)[::-1], rows=1,description='Startdatum:')
    widget_start_picker = widgets.DatePicker(description='Beginn Zeitraum', value = dt.datetime.now().date() - dt.timedelta(days=DEFAULT_DURATION_DAYS-1), style=wstyle)
    #widget_einheit = widgets.Select(options=['Stunden','Tage'], rows=1, value='Tage', continous_update=False, description='Einheit')
    
    widget_spreizung=widgets.Checkbox(value=False,description='Spreizungskurve anzeigen')
    widget_einstellungen=widgets.Checkbox(value=False,description='Heizungseinstellungen anzeigen')
    #wdg=widgets.interactive(plot_temperaturverlauf, df = widgets.fixed(df), start= widget_start_picker, dauer = widget_dauer, 
    #show_spreizung=widget_spreizung, show_heizungs_einstellungen=widget_einstellungen)
    #display(wdg)
    
    wdg=widgets.interactive_output(plot_temperaturverlauf, {'df': widgets.fixed(df), 'start': widget_start_picker, 'dauer': widget_dauer,
                                                            'show_spreizung' : widget_spreizung, 'show_heizungs_einstellungen': widget_einstellungen})
    hbox1 = widgets.HBox([widget_spreizung, widget_einstellungen])
    hbox2 = widgets.HBox([widget_start_picker, widget_dauer, widget_button])
    ui = widgets.VBox([hbox2, hbox1])
    display(ui,wdg)
    widget_button.on_click(on_button_clicked)


    

def get_heizkurve():
    df = None
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')

    #https://stackoverflow.com/questions/46576831/get-typeerror-index-must-be-datetimeindex-when-filtering-dataframe
    df.index = df['UnixTime']
    df1 = df.between_time('14:00', '18:00')
    df = df1
    df.to_csv('14_bis_15.csv', index = False, sep ='\t')

# main
display_temp_verlauf()
#get_heizkurve()

Output()